In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import pickle
import random
random.seed(42)
np.random.seed(42)

In [3]:
with open("../scripts/llm_dpo_ft_evaluated.pickle", "rb") as file:
#with open("../ressources/dpo_rl_01_24-12-24_s17_n600_e10_r100/dataset.pickle", "rb") as file:
#with open("../ressources/automode_evaluated_concat_s14s15s16_n300_24-12-18.pickle", "rb") as file:
#with open("../ressources/llm_evaluated_concat_s14n600_s15n600_train_2024-12-12_16.pickle", "rb") as file:
#with open("testdata.pkl", "rb") as file:
    original_df = pickle.load(file)

In [4]:
original_df

,level_0,index,description,configuration,parameters,argos,behavior_tree,scores,avg_score,llm_scores,llm_behavior_tree,llm_avg_score,llm_dpo_ft_scores,llm_dpo_ft_behavior_tree,llm_dpo_ft_avg_score
0,0,0,"In this setting, a circular arena with a radiu...","<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 --...,"[0.615385, 0.615385, 0.384615, 0.538462, 0.538...",0.607693,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0769231, 0.0, 0.0, 0.07...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,0.023077
1,1,1,The area is a rectangle with dimensions 4.76 x...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=RectangularArena(le...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 --...,"[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 2.0, ...",0.700000,[],--Root-- >>0--Condition--3--0.2422--0-- >>0--...,NaN,"[1.0, 0.0, 2.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,0.700000
2,2,2,The environment features a circle composed of ...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 --...,"[1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.300000,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,0.300000,"[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, ...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,0.600000
3,3,3,The environment is constructed as a rectangula...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=RectangularArena(le...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 --...,"[0.625, 1.0, 0.375, 0.625, 0.75, 0.75, 0.625, ...",0.700000,[],--Root-- -->Home-- -->Wait-- -->0-- -->0-...,NaN,[],--Root-- >>0--Condition--3 >>0.032--Condition...,NaN
4,4,4,The rectangular area has dimensions 3.38 m x 3...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=RectangularArena(le...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 --...,"[-6.01231, -5.57705, -4.33828, -4.06404, -5.82...",-4.943983,"[-10.8945, -9.60886, -10.9248, -11.0464, -10.7...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,-10.338910,"[-10.891, -10.7905, -9.70596, -10.5425, -10.72...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,-10.497951
5,5,5,The environment consists of a circular arena w...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",,None,NaN,[],--Root-- -->Action0-->Condition0-->Action1-->...,NaN,[],--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,NaN
6,6,6,The area is a rectangle with dimensions 6.76 x...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=RectangularArena(le...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 --...,"[-0.785453, -0.614938, -2.72515, -0.604966, -2...",-1.674051,"[-19.2398, -20.9475, -18.3469, -18.7683, -18.7...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,-16.203727,"[-6.71188, -7.98263, -6.15747, -4.42119, -6.40...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,-5.196720
7,7,7,"With a radius of 2.17 meters, the circular are...","<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 --...,"[-8.64841, -8.72471, -8.44374, -11.0059, -11.0...",-9.093409,"[-17.3721, -19.3659, -19.6226, -

In [5]:
df = original_df
df["type"] = df["parameters"].map(lambda x: type(x.objective_params).__name__)

In [6]:

# Step 2: Group by 'type' and 'dataset_position' and calculate the mean of the average scores
result = df.groupby(['type'])[['llm_dpo_ft_avg_score', 'llm_avg_score']].mean().reset_index()


# Display the results
print(result)

           type  llm_dpo_ft_avg_score  llm_avg_score
0   Aggregation              0.023077       0.000000
1    Connection            -10.497951     -10.338910
2  Distribution            -12.124975     -17.725909
3      Foraging              0.433333       0.433333
